In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Converting a Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Regular Expression

To implement the mathematical concept of "sets of sets" efficiently in TypeScript, we utilize the RecursiveSet library. This library allows sets to contain other sets as elements and supports using sets as keys in maps based on their structural content, which is essential for constructing the states of our deterministic machine.

In [ ]:
import { RecursiveSet, Tuple } from 'recursive-set';

State is the abstract type of the states of an Fsm.

In [ ]:
type State = string | number;

In [ ]:
type Char = string;

We represent the non-deterministic transition relation 
 by the following type:

In [ ]:
type TransRelDet = Map<string, RecursiveSet<State>>;

The deterministic finite state machine (DFA) produced in this notebook has the following type.

In [ ]:
type DFA = {
  Q: RecursiveSet<RecursiveSet<State>>;
  Sigma: RecursiveSet<Char>;
  delta: TransRelDet;
  q0: RecursiveSet<State>;
  A: RecursiveSet<RecursiveSet<State>>;
};

`RegExp` definiton of regular expression

In [ ]:
type BinaryOp = '⋅' | '+';
type UnaryOp = '*';

In [ ]:
type RegExpContent = RegExp | BinaryOp | UnaryOp;

In [ ]:
type RegExp = 
  | number 
  | string 
  | Tuple<[RegExp, UnaryOp]> 
  | Tuple<[RegExp, BinaryOp, RegExp]>;

First, we define `key`, a helper function to generate unique identifiers for transition lookups given a state (or a set of states) and a character.

In [ ]:
function key(q: State | RecursiveSet<State>, c: Char): string {
  return `${q.toString()},${c}`;
}

The function `regexp_sum` takes a set $S = \{ r_1, \cdots, r_n \}$ of regular expressions
as its argument.  It returns the regular expression 
$$ r_1 + \cdots + r_n. $$
The regular expression will be represented as a nested tuple that uses the operators `+` (for alternatives), `&` (for concatenations), and `*` (for repetitions).


In [ ]:
function regexpSum(S: RecursiveSet<RegExp> | RegExp[]): RegExp {
  const elems: RegExp[] = Array.isArray(S) ? S : (Array.from(S) as RegExp[]);
  const n = elems.length;

  if (n === 0) return 0;
  if (n === 1) return elems[0];

  const [r, ...rest] = elems;

  // Explizite Angabe der Tupel-Struktur [RegExp, BinaryOp, RegExp]
  return new Tuple<[RegExp, BinaryOp, RegExp]>(
    r, 
    '+' as BinaryOp, 
    regexpSum(rest)
  );
}

The function `rpq` assumes there is some <span style="font-variant:small-caps;">Fsm</span>
$$ F = \langle \texttt{States}, \Sigma, \delta, \texttt{q0}, \texttt{Accepting} \rangle $$
given and takes five arguments:
- `p1` and `p2` are states of the <span style="font-variant:small-caps;">Fsm</span> $F$,
- $\Sigma$ is the alphabet of the <span style="font-variant:small-caps;">Fsm</span>,
- $\delta$ is the transition function of the <span style="font-variant:small-caps;">Fsm</span> $F$, and
- `Allowed` is a subset of the set `States`.  On recursive calls, `Allowed` is a list of states.

The function `rpq` computes a regular expression that describes those strings that take the 
<span style="font-variant:small-caps;">Fsm</span> $F$ from the state `p1` to state `p2`.
When $F$ switches states from `p1` to `p2` only states in the set `Allowed` may be visited in-between the states `p1` and `p2`.

The function is defined by recursion on the set `Allowed`.  There are two cases
- $\texttt{Allowed} = \{\}$.  
  Define `AllChars`as the set of all characters that when read by $F$ in the state `p1` cause $F$ to enter the state `p2`:
  $$ \texttt{AllChars} = \{ c \in \Sigma \mid \delta(p_1, c) = p_2 \} $$
  Then we need a further case distinction:
  - $p_1 = p_2$: In this case we have:
    $$ \texttt{rpq}(p_1, p_2, \{\}) := \sum\limits_{c\in\texttt{AllChars}} c \quad + \varepsilon$$
    If $\texttt{AllChars} = \{\}$ the sum $\sum\limits_{c\in\texttt{AllChars}} c$ is to be interpreted as the
    regular expression $\emptyset$ that denotes the empty language. 
    
    Otherwise, if $\texttt{AllChars} = \{c_1,\cdots,c_n\}$ we have
    $\sum\limits_{c\in\texttt{AllChars}} c \quad = c_1 + \cdots + c_n$.
  - $p_1 \not= p_2$: In this case we have:
    $$ \texttt{rpq}(p_1, p_2, \{\}) := \sum\limits_{c\in\texttt{AllChars}} c \quad$$
- $\texttt{Allowed} = \{ q \} \cup \texttt{RestAllowed}$.  In this case we recursively define the following variables:
  1. $\texttt{rp1p2} := \texttt{rpq}(p_1, p_2, \Sigma, \delta, \texttt{RestAllowed})$,
  2. $\texttt{rp1q } := \texttt{rpq}(p_1, q, \Sigma, \delta, \texttt{RestAllowed})$,
  3. $\texttt{rqq }\texttt{ } := \texttt{rpq}(q, q, \Sigma, \delta, \texttt{RestAllowed})$,
  4. $\texttt{rqp2 } := \texttt{rpq}(q, p_2, \Sigma, \delta, \texttt{RestAllowed})$.

  Then we can define:
  $$ \texttt{rpq}(p_1, p_2, \texttt{Allowed}) := \texttt{rp1p2} + \texttt{rp1q} \cdot \texttt{rqq}^* \cdot \texttt{rqp2} $$
  This formula can be understood as follows:  If a string $w$ is read in state $p_1$ and reading this string takes the 
  <span style="font-variant:small-caps;">Fsm</span> $F$ from the state $p_1$ to the state $p_2$ while only visiting states from the set 
  `Allowed` in-between, then there are two cases:
  - Reading $w$ does not visit the state $q$ in-between.  Hence the string $w$ can be described by the regular expression
    `rp1p2`.
  - The string $w$ can be written as $w = t u_1 \cdots u_n v$ where:
    - reading $t$ in the state $p_1$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ into the state $q$,
    - for all $i \in \{1,\cdots,n\}$ reading $v_i$ in the state $q$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ from $q$ to $q$, and
    - reading $v$ in the state $q$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ into the state $p_2$.

In [ ]:
function rpq(
  p1: RecursiveSet<State>,
  p2: RecursiveSet<State>,
  Sigma: RecursiveSet<Char>,
  delta: TransRelDet,
  Allowed: RecursiveSet<State>[]
): RegExp {
  if (Allowed.length === 0) {
    const allChars = new RecursiveSet<Char>();
    for (const symbol of Sigma) {
      const c = symbol as Char;
      const target = delta.get(key(p1, c));
      if (target && (target === p2 || target.equals(p2))) {
        allChars.add(c);
      }
    }

    const r = regexpSum(allChars);

    if (p1.equals(p2)) {
      // ε + r
      return new Tuple<[RegExp, BinaryOp, RegExp]>(
        'ε', 
        '+' as BinaryOp, 
        r
      );
    } else {
      return r;
    }
  }

  const [q, ...RestAllowed] = Allowed;

  const rp1p2 = rpq(p1, p2, Sigma, delta, RestAllowed);
  const rp1q  = rpq(p1, q, Sigma, delta, RestAllowed);
  const rqq   = rpq(q, q, Sigma, delta, RestAllowed);
  const rqp2  = rpq(q, p2, Sigma, delta, RestAllowed);

  // Konstruktion: (rp1q ⋅ rqq* ⋅ rqp2)

  // 1. Schleife: rqq* -> Struktur [RegExp, UnaryOp]
  const loop = new Tuple<[RegExp, UnaryOp]>(
    rqq, 
    '*' as UnaryOp
  );

  // 2. Konkatenation 1: rp1q ⋅ loop -> Struktur [RegExp, BinaryOp, RegExp]
  // Hinweis: 'loop' ist ein RegExp, da Tuple<...> ein RegExp ist.
  const concat1 = new Tuple<[RegExp, BinaryOp, RegExp]>(
    rp1q, 
    '⋅' as BinaryOp, 
    loop
  );

  // 3. Konkatenation 2: concat1 ⋅ rqp2
  const concat2 = new Tuple<[RegExp, BinaryOp, RegExp]>(
    concat1, 
    '⋅' as BinaryOp, 
    rqp2
  );

  // Gesamtergebnis: rp1p2 + concat2
  return new Tuple<[RegExp, BinaryOp, RegExp]>(
    rp1p2, 
    '+' as BinaryOp, 
    concat2
  );
}

The function `dfa_2_regexp` takes a deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ and computes a regular expression $r$ that describes the same language as $F$, i.e. we have
$$ L(A) = L(r). $$

In [ ]:
function dfa2regexp(F: DFA): RegExp {
  const { Q, Sigma, delta, q0, A } = F;
  const allStates = Array.from(Q) as RecursiveSet<State>[];
  const parts = new RecursiveSet<RegExp>();

  for (const acc of A) {
    const p = acc as RecursiveSet<State>;
    const r = rpq(q0, p, Sigma, delta, allStates);
    parts.add(r);
  }

  return regexpSum(parts);
}

The notebook `06-Test-DFA-2-RegExp.ipynb` provides a test for the function `dfa_2_regexp`.